# We show how to run rigid and/or piecewise rigid motion correction on the demoMovie.tif dataset found in the datasets folder of this repo

In [18]:
import numpy as np

import jnormcorre
import jnormcorre.motion_correction
import jnormcorre.utils.registrationarrays as registrationarrays
from jnormcorre.utils.registrationarrays import TiffArray

import matplotlib.pyplot as plt
import os
import tifffile
%load_ext autoreload


# Specify dataset

## The below code loads the full dataset into memory. Any object that implements the lazy data loader API: https://jnormcorre.readthedocs.io/en/latest/api.html#jnormcorre-utils-lazy-array-api can be used here. This is very convenient for processing datasets that are too big to fit into RAM. Existing support exists for some 

In [4]:
filename = "../datasets/demoMovie.tif"

#####
##Run this if you want to load the dataset into RAM for processing
#####

data = tifffile.imread(filename).astype("float")


#####
##If instead you want to define a lazy data loader, run the line below.
#####
# data = TiffArray(filename)

# Run Motion Correction. See the params here https://jnormcorre.readthedocs.io/en/latest/api.html#motioncorrect-class for reference

In [4]:
## Rigid motion correction parameters
max_shifts = (5, 5)
frames_per_split = 500
num_splits_to_process_rig = 5
niter_rig = 4
save_movie = False


'''
Parameters for estimating a piecewise rigid template 
Set pw_rigid = False if you do not want to to piecewise motion correction)
'''
pw_rigid = True
strides = (50, 50)
overlaps = (5, 5)
max_deviation_rigid = 5

'''
1P Params (set this to a high pass filter kernel size  if you want to run the 1p motion correction pipeline)
Examples: gSig_filt = (5, 5)
'''
gSig_filt = None

corrector = jnormcorre.motion_correction.MotionCorrect(data, max_shifts=max_shifts, frames_per_split=frames_per_split,
                                                num_splits_to_process_rig=num_splits_to_process_rig, strides=strides,
                                                       overlaps=overlaps, 
                                                       max_deviation_rigid = max_deviation_rigid, niter_rig=niter_rig,
                                                       pw_rigid = pw_rigid, gSig_filt=gSig_filt)


frame_corrector, output_file = corrector.motion_correct(
    template=None, save_movie=save_movie
)

# Use the frame_corrector and the data variable to define a registration array; this is useful for treating the motion corrected output like any other standard np.ndarray

In [ ]:
frame_corrector.batching = 100 #Make this smaller or larger as needed to avoid GPU errors
motion_correction_dataloader = registrationarrays.RegistrationArray(frame_corrector, data, pw_rigid=False)

# Generate Visualization: We can now directly generate side-by-side visualizations of the raw and motion corrected data

## Option 1: Write the results to a .tiff file

In [ ]:
time_slice = slice(0, 500, 1) #Change this to reflect which frames you want to compare
raw_data_slice = data[time_slice]
motion_corrected_data_slice = motion_correction_dataloader[time_slice]

output = np.concatenate([raw_data_slice, motion_corrected_data_slice], axis = 2)

tifffile.imwrite("diagnostic.tiff", output)

# Option 2: (Recommended) If you have fastplotlib installed (see https://github.com/fastplotlib/fastplotlib/tree/main) you can use ImageWidget to interactively view the results in the notebook. 

In [9]:
from fastplotlib.widgets import ImageWidget
iw = ImageWidget(data = [data, motion_correction_dataloader], 
                 names=["Raw", "Motion Corrected"], 
                 histogram_widget = True)

iw.show()